In [ ]:
%run configs.ipynb
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import requests
import time
import re
import datetime
import pandas as pd
import csv
base='https://www.instagram.com'


# Project 1 begins ...

In [ ]:
driver=webdriver.Chrome(executable_path='C:/Users/HP/Desktop/chromedriver_win32 (1)/chromedriver')
driver.maximize_window()

In [ ]:
driver.get('https://www.instagram.com/')

# 1. Social Login Using Facebook

In [ ]:
#social login using facebook
def login(driver):
    facebookicon=driver.find_element_by_class_name('KPnG0')
    facebookicon.click()
    ## explicit wait
    ## create object of class WebDriver wait
    wait=WebDriverWait(driver,15)
    wait.until(EC.presence_of_element_located((By.NAME,'email')))

    email=driver.find_element_by_name('email')
    password=driver.find_element_by_name('pass')
    email.click()
    email.clear()
    email.send_keys(useremail) #SAMPLE USEREMAIL
    password.click()
    password.clear()
    password.send_keys(userpassword) # SAMPLE PASSWORD
    email.submit()

In [ ]:
login(driver)

# Search a Word

In [ ]:
# To search
def searchWord(driver,word):
    div=driver.find_element_by_class_name('eyXLr')
    div.click()
    input=driver.find_element_by_class_name('XTCLo')
    input.clear()
    input.send_keys(word)
    
    

# Close Search Field

In [ ]:
# to close the search field
def closeSearchField(driver):
    try:
        div=driver.find_element_by_class_name('aIYm8')
        div.click()
    except:
        pass

In [ ]:
searchWord(driver,'food')

# 2. Searching and Printing the list appeared 

In [ ]:
# to print the handle list obtained after searching the word
def printHandleList(driver,key):
    searchWord(driver,key)  # searches word
    time.sleep(5)
    handle_list=driver.find_elements_by_class_name('yCE8d')
    root=BeautifulSoup(driver.page_source,'lxml')
    elements=root.find_all('a',class_='yCE8d')
    handleSet=[]
    for ele in elements:
        if not ele.find('span',class_='coreSpriteHashtag'):
            handleSet.append(ele.find('span',class_='Ap253').string)
#             print(ele.find('span',class_='Ap253').string)
    
    return handleSet


In [ ]:
# Type for “food” in search bar and print all the names of the Instagram Handles that are displayed in list after typing “food”
# Note : Make sure to avoid printing hashtags

print(printHandleList(driver,'food'))

# 3. Search And Open Handler

In [ ]:
def searchAndOpenHandle(driver,handleName):
    searchWord(driver,handleName)
    wait=WebDriverWait(driver,10)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME,'yCE8d')))
    handle_list=driver.find_elements_by_class_name('yCE8d')
    root=BeautifulSoup(driver.page_source,'lxml')
    elements=root.find_all('a',class_='yCE8d')
    links=set()
    for i in range(len(elements)):
        if(elements[i].find(class_='Ap253').string==handleName):
            handle_list[i].click()
            break
    closeSearchField(driver)
    

In [ ]:
# Searching and Opening a profile using 
# Open profile of “sodelhi” 
searchAndOpenHandle(driver,'sodelhi')

# 4.1 Follow a handler

In [ ]:
def follow(driver,handleName):
    searchAndOpenHandle(driver,handleName)
    try :
        driver.find_element_by_class_name('glyphsSpriteFriend_Follow')
        print('Already Following')
    except NoSuchElementException :
        driver.find_element_by_class_name('_5f5mN').click()
        
    
         

# 4.2 Unfollow a handler

In [ ]:
def unfollow(driver,handleName):
    searchAndOpenHandle(driver,handleName)
    try :
        ele=driver.find_element_by_class_name('glyphsSpriteFriend_Follow')
        ele.click()
        wait=WebDriverWait(driver,10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME,'-Cab_')))
        unfollow=driver.find_element_by_class_name('-Cab_')
        unfollow.click()
    except NoSuchElementException :
        print('Already Unfollowing')
        

In [ ]:
# Open the Instagram Handle of “So Delhi”
# Start following it. Print a message if you are already following
follow(driver,'sodelhi')

In [ ]:
follow(driver,'sodelhi')

In [ ]:
# unfollow the instagram handle. Print a message if you have already unfollowed.
unfollow(driver,'sodelhi')

# 5.1 Like top 30 posts 

In [ ]:
def likePosts(driver,handleName):
    searchAndOpenHandle(driver,handleName)
    time.sleep(5)
    count=0
    flag=0
    while True:
        bodyHeight=driver.execute_script('return document.body.scrollHeight;')
#         print(bodyHeight)
        posts=driver.find_elements_by_class_name('_9AhH0')
        for post in posts[count:]:
            post.click()
            wait=WebDriverWait(driver,10)
#             wait.until(EC.presence_of_element_located((By.CLASS_NAME,'fr66n')))
            wait.until(EC.visibility_of_element_located((By.XPATH,'//span[@class="fr66n"]//*[name()="svg"]')))
#             heart=driver.find_element_by_xpath('//span[@class="fr66n")]//svg[@class="_8-yf5"]')
            heart=driver.find_element_by_xpath('//span[@class="fr66n"]//*[name()="svg"]')
#             print(heart.get_attribute('aria-label'))
            if heart.get_attribute('aria-label')=='Unlike':
                print('already liked')
            else:
                heart.click()
            close= driver.find_element_by_xpath('//div[contains(@class,"yiMZG")]//*[name()="svg"]')
#             driver.find_element_by_class_name('yiMZG')
            close.click()
            count=count+1
#             print(count)
            if count==30:
                flag=1
                break
        if flag==1:
            break
        else:
            driver.execute_script('window.scrollTo(0,arguments[0]);',bodyHeight)


In [ ]:
likePosts(driver,'dilsefoodie')

# 5.2 UnLike top 30 posts 

In [ ]:
def unlikePosts(driver,handleName):
    searchAndOpenHandle(driver,handleName)
    time.sleep(5)
    count=0
    flag=0
    while True:
        bodyHeight=driver.execute_script('return document.body.scrollHeight;')
#         print(bodyHeight)
        posts=driver.find_elements_by_class_name('_9AhH0')
        for post in posts[count:]:
            post.click()
            wait=WebDriverWait(driver,10)
            wait.until(EC.visibility_of_element_located((By.XPATH,'//span[@class="fr66n"]//*[name()="svg"]')))
            heart=driver.find_element_by_xpath('//span[@class="fr66n"]//*[name()="svg"]')
            if heart.get_attribute('aria-label')=='Like':
                print('already unliked')
            else:
                heart.click()
            close= driver.find_element_by_xpath('//div[contains(@class,"yiMZG")]//*[name()="svg"]')
            close.click()
            count=count+1
            if count==30:
                flag=1
                break
        if flag==1:
            break
        else:
            driver.execute_script('window.scrollTo(0,arguments[0]);',bodyHeight)


In [ ]:
unlikePosts(driver,'dilsefoodie')

# 6. Extract list of followers

In [ ]:
def listFollowers(driver,handleName):
    searchAndOpenHandle(driver,handleName)
    time.sleep(2)
    follower=driver.find_element_by_partial_link_text(' followers')
    follower.click()
    
    wait=WebDriverWait(driver,10)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME,'sqdOP')))
    prevHeight=0
    bodyHeight=driver.execute_script('return document.getElementsByClassName("isgrP")[0].scrollHeight;')
    
    count=0
    followers=set() # who are users followers
    following=set() # who the user follows
    while True:
        
        flag=0
        bodyHeight=driver.execute_script('return document.getElementsByClassName("isgrP")[0].scrollHeight;')
        # if there are less than 500 followers
        if bodyHeight==prevHeight:
            close=driver.find_element_by_xpath('//div[@class="WaOAr"]//*[name()="svg"]')
            close.click()
            return (followers,following)
        followButtons=driver.find_elements_by_xpath('//div[@class="isgrP"]//button[contains(@class,"sqdOP")]')
        usernames=driver.find_elements_by_class_name('FPmhX')
        
        for i in range(count,len(followButtons)):
            if followButtons[i].get_attribute('innerHTML')=='Follow':
                followers.add(usernames[i].get_attribute('innerHTML'))
            else:
                followers.add(usernames[i].get_attribute('innerHTML'))
                following.add(usernames[i].get_attribute('innerHTML'))
            
            count=count+1
            if count==500:
                flag=1
                break
                
        if flag==1:
            close=driver.find_element_by_xpath('//div[@class="WaOAr"]//*[name()="svg"]')
            close.click()
            return (followers,following)
            break
        else:
            prevHeight=bodyHeight
            driver.execute_script('document.getElementsByClassName("isgrP")[0].scrollTo(0,arguments[0]);',bodyHeight)
            time.sleep(5)
            

In [ ]:
#Extract the usernames of the first 500 followers of ‘foodtalkindia’ and ‘sodelhi’.
# Now print all the followers of “foodtalkindia” that you are following

foodtalkindia=listFollowers(driver,'foodtalkindia')
print(foodtalkindia[0],foodtalkindia[1])
sodelhi=listFollowers(driver,'sodelhi')
print(sodelhi[0],sodelhi[1])
user=listFollowers(driver,'paruprerna')
print(user[0],user[1])


In [ ]:
#  print all the followers of “foodtalkindia” that you are following but those who don’t follow you.
print(user[0].intersection(foodtalkindia[1]))


# 7. Story

In [ ]:
# Check the story of ‘coding.ninjas’. Consider the following Scenarios and print error messages accordingly -
# If You have already seen the story.
# Or The user has no story.
# Or View the story if not yet seen.
def storyCheck(driver,handleName):
    searchAndOpenHandle(driver,handleName)
    time.sleep(5)
    if driver.find_element_by_css_selector('div.RR-M- canvas').get_attribute('height')=='208':
        print('You have already seen the story')
    elif driver.find_element_by_css_selector('div.RR-M- canvas').get_attribute('height')=='210':
        driver.find_element_by_class_name('RR-M-').click()
        try:
            WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME,"_8XqED")))
            is_status=driver.find_element_by_class_name('_8XqED')
            print('story not seen')
        except:
            print('no story')
    
            

In [ ]:
storyCheck(driver,'coding.ninjas')

# Project 2nd begins..........


# Top 10 handlers with their followers number as a dictionary
1.Now your friend has followed a lot of different food bloggers, he needs to analyse the habits of these bloggers.
From the list of instagram handles you obtained when you searched ‘food’ in previous project. Open the first 10 handles and find the top 5 which have the highest number of followers
Now Find the number of posts these handles have done in the previous 3 days.
Depict this information using a suitable graph.

In [ ]:

def topHandles(key):
    handles=printHandleList(driver,key)
    closeSearchField(driver)
    count=0
    hdict={}
    for handle in handles:
        searchAndOpenHandle(driver,handle)
        time.sleep(3)
        followers=driver.find_element_by_partial_link_text('followers')
        soup=BeautifulSoup(followers.get_attribute('innerHTML'),'lxml')
        num=re.sub(',',"",soup.span["title"])
        hdict[handle]=int(re.search('\d+',num).group())
        count=count+1
        if count==10:
            break
    
    return hdict
    

In [ ]:
res=topHandles('food')
# to find the top 5 handlers with highest number of followers
dataf=pd.DataFrame(list(res.values()),list(res.keys()))
dataf=dataf.sort_values(by=[0],ascending=False)
print(dataf.index[0:5])



# number of posts in last 3 days

In [ ]:
def numberOfPosts(handles):
    postdict={}
    for handle in handles:
        searchAndOpenHandle(driver,handle)
        time.sleep(5)
        flag=0
        postcount=0
        prevHeight=0
        while True:
            bodyHeight=driver.execute_script('return document.body.scrollHeight;')
            # if all posts are visited
            if bodyHeight==prevHeight:
                postdict[handle]=postcount
                close=driver.find_element_by_xpath('//div[@class="WaOAr"]//*[name()="svg"]')
                close.click()
                return
            posts=driver.find_elements_by_class_name('_9AhH0')
            count=0
            for post in posts[count:]:
                post.click()
                wait=WebDriverWait(driver,10)
                wait.until(EC.visibility_of_element_located((By.CLASS_NAME,'_1o9PC')))
                t=driver.find_element_by_class_name('_1o9PC')
                d1=datetime.datetime.now()
                z=t
                t=t.get_attribute('datetime').split('T')[0].split('-')
                z=z.get_attribute('datetime').split('T')[1].split('Z')[0].split(':')
                d2=datetime.datetime(int(t[0]),int(t[1]),int(t[2]),int(z[0]),int(z[1]))
#                 print((d1-d2).days)
                close= driver.find_element_by_xpath('//div[contains(@class,"yiMZG")]//*[name()="svg"]')
                close.click()
#                 postcount=postcount+1
                count=count+1
                if (d1-d2).days>3:
                    flag=1
                    break
                else:
                    postcount=postcount+1
            if flag==1:
                postdict[handle]=postcount
                break
            else:
                driver.execute_script('window.scrollTo(0,arguments[0]);',bodyHeight)

    
    return postdict
    

In [ ]:
postdict=numberOfPosts(dataf.index[0:5])

In [ ]:
print(postdict)

In [ ]:
# bubble graph
# x --> handlerName
# y --> number of follwers
# bubble size --> number of posts done in last 3 days
plt.scatter(dataf.index[0:5],dataf.values[0:5],s=list(postdict.values()),c="r") 
plt.xticks(rotation=40)

# Scrape contents of top 10 posts 
Open the 5 handles you obtained in the last question, and scrape the content of the first 10 posts of each handle.
Prepare a list of all words used in all the scraped posts and calculate the frequency of each word.
Create a csv file with two columns : the word and its frequency
Now, find the hashtags that were most popular among these bloggers
Plot a Pie Chart of the top 5 hashtags obtained and the number of times they were used by these bloggers in the scraped posts.

In [ ]:
def scrapeContents(handles):
    contents=[]
    for handle in handles:
        searchAndOpenHandle(driver,handle)
        time.sleep(5)
        flag=0
        postcount=0
        prevHeight=0
        count=0
        posts=driver.find_elements_by_class_name('_9AhH0')
        for post in posts[0:10]:
            post.click()
            wait=WebDriverWait(driver,10)
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME,'C4VMK')))
            content=driver.find_element_by_class_name('C4VMK')
            contents.append(BeautifulSoup(content.get_attribute('innerHTML'),'html.parser'))
            close= driver.find_element_by_xpath('//div[contains(@class,"yiMZG")]//*[name()="svg"]')
            close.click()
    return contents
    
    

In [ ]:
wordsdict={}
wordslist=[]
data=scrapeContents(dataf.index[0:5])
for d in data:
    for i in d.contents[1].strings:
        for j in i.split(" "):
            if len(j)!=0:
                wordsdict[j]=wordsdict.get(j,0)+1
                



In [ ]:
# convert to pandas then convert to csv file
words_df=pd.DataFrame(list(wordsdict.items()),columns=['word','frequency'])
print(words_df)
words_df.to_csv('words.csv',index=False)

In [ ]:
df=pd.read_csv('words.csv',encoding='utf-8')
hashtagdict={}
# to find all the hash tags and hahtagdict dictionary is created
for i in range(len(df['word'])):
    if(df.iloc[i]['word'].startswith('#')):
        hashtagdict[df.iloc[i]['word']]=df.iloc[i]['frequency']
print("hash tag dict ----> ")
print(hashtagdict)
hashf=pd.DataFrame(list(hashtagdict.values()),list(hashtagdict.keys()))
hashf=hashf.sort_values(by=[0],ascending=False)
# sorted pandas dataframe for hashtags
print(hashf)

In [ ]:
# labels are hashf dataframe indexes(handler name) and percentage are based on frequency of these hashtags
print("Top five hashtags ---->",hashf.index[0:5])
plt.pie(hashf[0:5].values, labels=hashf.index[0:5], autopct="%.2f%%",counterclock=False,  startangle=100)
plt.show()

# scrape likes 
You need to also calculate average followers : likes ratio for the obtained handles.
Followers : Likes ratio is calculated as follows:
Find out the likes of the top 10 posts of the 5 handles obtained earlier.
Calculate the average likes for a handle.
Divide the average likes obtained from the number of followers of the handle to get the average followers:like ratio of each handle.
Create a bar graph to depict the above obtained information.

In [ ]:
def scrapeLikes(handles):
#     contents=[]
    avgLikes={}
    for handle in handles:
        searchAndOpenHandle(driver,handle)
        time.sleep(5)
        flag=0
        postcount=0
        prevHeight=0
        count=0
        posts=driver.find_elements_by_class_name('_9AhH0')
        total=0
        for post in posts[0:10]:
            post.click()
            wait=WebDriverWait(driver,10)
            try:
                wait = WebDriverWait(driver, 6) #Waiting for 10 seconds
                wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'Nm9Fw')]/button/span")))
                likeFind = driver.find_element_by_xpath('//div[contains(@class, "Nm9Fw")]/button/span')#
                currentLikes = likeFind.text
            except:
                wait = WebDriverWait(driver, 10) #Waiting for 10 seconds
                wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "vcOH2")))
                viewFind = driver.find_element_by_class_name('vcOH2')
                viewFind.click()
                wait = WebDriverWait(driver, 10) #Waiting for 10 seconds
                wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'vJRqr')]/span")))
                likeFind = driver.find_element_by_xpath('//div[contains(@class, "vJRqr")]/span')#
                currentLikes = likeFind.text
                body = driver.find_element_by_class_name('QhbhU')#QhbhU
                body.click()
#             print(currentLikes)
            currentLikes=re.sub(',',"",currentLikes)
#             print(int(re.search('\d+',currentLikes).group()))
            total=total+int(currentLikes)
            
#             total.append(BeautifulSoup(like.get_attribute('innerHTML'),'html.parser'))

#             print(BeautifulSoup(like.get_attribute('innerHTML'),'html.parser'))
            close= driver.find_element_by_xpath('//div[contains(@class,"yiMZG")]//*[name()="svg"]')
            close.click()
        avgLikes[handle]=total
    return avgLikes

In [ ]:
likedict=scrapeLikes(dataf.index[0:5])
print(likedict)

In [ ]:
# print(dataf) # Top 5 handlers with followers 
# print(likedict) # likes dict
ratiodict={}
for handle in dataf.index[0:5]:
    ratiodict[handle]=dataf[0].loc[handle]/(likedict[handle]/10)
    
plt.bar(ratiodict.keys(),ratiodict.values())
plt.xticks(rotation=40)
plt.show()

print(ratiodict)
